# 1. SETTINGS

In [2]:
# libraries
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import date
import matplotlib.pyplot as plt
from IPython.display import display
import scipy.stats

In [3]:
# pandas options
pd.set_option("display.max_columns", None)

In [4]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
# garbage collection
import gc
gc.enable()

# 2. FUNCTIONS

In [6]:
### FUNCTION FOR COUNTING MISSINGS
def count_missings(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending = False)
    table = pd.concat([total, percent], axis = 1, keys = ["Total", "Percent"])
    table = table[table["Total"] > 0]
    return table

# 3. DATA IMPORT

In [7]:
# import datasets
test  = pd.read_csv("../data/raw/Challenge_20180423.csv")
cust  = pd.read_csv("../data/raw/Customer.csv")
bond  = pd.read_csv("../data/raw/Isin.csv")
markt = pd.read_csv("../data/raw/Market.csv")
macro = pd.read_csv("../data/raw/MarketData_Macro.csv")
trade = pd.read_csv("../data/raw/Trade.csv")

In [43]:
# check all datasets
display(test.head(3))
print("Test data:", test.shape)
print("------------------------------")
display(cust.head(3))
print("Customer data:", cust.shape)
print("------------------------------")
display(bond.head(3))
print("Bonds data:", bond.shape)
print("------------------------------")
display(markt.head(3))
print("Market data:", markt.shape)
print("------------------------------")
display(macro.head(3))
print("Macroeconomic data:", macro.shape)
print("------------------------------")
display(trade.head(3))
print("Trade data:", trade.shape)

,PredictionIdx,DateKey,CustomerIdx,IsinIdx,BuySell,CustomerInterest
0,a1e0d80784,20180423,1856,13323,Buy,NaN
1,c2cc6cc2a8,20180423,1856,9230,Buy,NaN
2,a8e94f6344,20180423,1780,9157,Buy,NaN


Test data: (484758, 6)
------------------------------


,CustomerIdx,Sector,Subsector,Region,Country
0,2975,Banks and Intermediaries,Bank,Americas,BARBADOS
1,1594,Asset Managers & Hedge Funds,NaN,Americas,BERMUDA
2,399,Corporation,Corp - Comm. & Prof. Services,Americas,BERMUDA


Customer data: (3471, 5)
------------------------------


,IsinIdx,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
0,0,238,20381231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.246002e+09,STEP CPN
1,1,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED
2,2,238,20331231,20051129,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.501245e+10,FIXED


...

Market data: (9867747, 5)
------------------------------


,DateKey,SSE,DAX,EUROSTOXX,VSTOXX,FTSE100,HSI,NIKKEI,DOWJONES_INDU,SP500,VIX,FX_USD.ARS,FX_USD.AUD,FX_USD.BRL,FX_USD.CAD,FX_USD.CHF,FX_USD.CNO,FX_USD.CNY,FX_USD.EUR,FX_USD.GBP,FX_USD.HKD,FX_USD.IDR,FX_USD.JPY,FX_USD.NOK,FX_USD.SGD,FX_USD.TRY,FX_USD.ZAR,MoneyMarket_ARS3M,MoneyMarket_AUD3M,MoneyMarket_CAD3M,MoneyMarket_CHF3M,MoneyMarket_CNO3M,MoneyMarket_CNY3M,MoneyMarket_EUR3M,MoneyMarket_GBP3M,MoneyMarket_HKD3M,MoneyMarket_IDR3M,MoneyMarket_JPY3M,MoneyMarket_NOK3M,MoneyMarket_SGD3M,MoneyMarket_TRY3M,MoneyMarket_USD3M,MoneyMarket_ZAR3M,Swap_ARS10Y,Swap_ARS2Y,Swap_ARS5Y,Swap_AUD10Y,Swap_AUD2Y,Swap_AUD30Y,Swap_AUD5Y,Swap_BRL10Y,Swap_BRL2Y,Swap_BRL5Y,Swap_CAD10Y,Swap_CAD2Y,Swap_CAD30Y,Swap_CAD5Y,Swap_CHF10Y,Swap_CHF2Y,Swap_CHF30Y,Swap_CHF5Y,Swap_CNH10Y,Swap_CNH2Y,Swap_CNH30Y,Swap_CNH5Y,Swap_CNO10Y,Swap_CNO2Y,Swap_CNO30Y,Swap_CNO5Y,Swap_CNY10Y,Swap_CNY2Y,Swap_CNY30Y,Swap_CNY5Y,Swap_EUR10Y,Swap_EUR2Y,Swap_EUR30Y,Swap_EUR5Y,Swap_GBP10Y,Swap_GBP2Y,Swap_GBP30Y,Swap_GBP5Y,Swap_HKD10Y,Swap_HKD2Y,Swap_HKD30Y,Swap_HKD5Y,Swap_IDR10Y,Swap_IDR2Y,Swap_IDR30Y,Swap_IDR5Y,Swap_JPY10Y,Swap_JPY2Y,Swap_JPY30Y,Swap_JPY5Y,Swap_NOK10Y,Swap_NOK2Y,Swap_NOK30Y,Swap_NOK5Y,Swap_SGD10Y,Swap_SGD2Y,Swap_SGD30Y,Swap_SGD5Y,Swap_TRY10Y,Swap_TRY2Y,Swap_TRY5Y,Swap_USD10Y,Swap_USD2Y,Swap_USD30Y,Swap_USD5Y,Swap_ZAR10Y,Swap_ZAR2Y,Swap_ZAR30Y,Swap_ZAR5Y
0,20150101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.20,0.116959,0.81620,0.377658,0.861030,1.006264,0.161512,0.163399,1.2101,1.55775,0.128941,0.000081,0.008341,0.133451,0.756287,0.428964,0.086457,35.230452,2.724540,1.303309,-0.063143,3.701588,3.617925,0.077487,0.561227,0.381829,7.17183,0.112078,1.459648,0.709986,10.001685,0.256570,6.130798,19.989643,29.980614,21.985733,3.041150,2.398950,3.625076,2.558248,NaN,NaN,NaN,2.277279,NaN,2.760043,1.775859,0.409878,NaN,1.082563,-0.046046,3.749868,3.418686,NaN,3.549519,3.750081,3.419989,NaN,3.550031,3.439794,3.218679,NaN,3.239508,0.697389,NaN,1.408397,0.233572,NaN,NaN,NaN,NaN,2.252274,0.95467,NaN,1.814259,8.955845,8.368177,9.552123,8.708471,0.444559,NaN,1.233032,0.180984,1.806932,NaN,2.179804,1.295793,NaN,NaN,NaN,NaN,9.885583,10.069276,9.915120,2.295037,NaN,2.712112,1.774452,7.963276,6.856526,8.331874,7.403867
1,20150102,NaN,9764.73,3139.44,26.2531,6547.80,23857.82,NaN,17832.99,2058.20,17.79,0.116891,0.81140,0.371292,0.851136,0.998851,0.161186,0.163292,1.2008,1.53340,0.128934,0.000080,0.008308,0.131841,0.752757,0.427241,0.085609,33.000000,2.736281,1.302850,-0.063000,3.701588,3.617925,0.077179,0.563380,0.385000,7.10000,0.112078,1.490000,0.790000,10.020000,0.255218,6.125000,20.000000,30.000000,22.000000,3.108394,2.452905,3.692179,2.625847,11.767860,12.807338,12.202677,2.231223,1.445105,2.716551,1.748069,0.367391,NaN,1.040147,-0.058452,3.750081,3.419989,NaN,3.550031,3.750081,3.419989,NaN,3.550031,3.439794,3.218679,NaN,3.239508,0.640191,0.062120,1.347191,0.211268,1.674890,0.810358,2.109754,1.305976,2.290000,0.97500,2.75196,1.840000,8.950000,8.350000,9.550000,8.700000,0.444559,NaN,1.233032,0.180984,1.794443,1.100220,2.168744,1.280511,2.447394,1.132505,2.95190,1.948474,9.946260,10.151881,9.976101,2.236910,0.893242,2.647499,1.746344,7.940000,6.840000,8.310000,7.390000
2,20150105,3350.519,9473.16,3023.14,29.6236,6417.16,23721.32,17408.71,17501.65,2020.58,19.92,0.116782,0.80905,0.366757,0.850268,0.993764,0.160822,0.163265,1.1939,1.52555,0.128926,0.000079,0.008366,0.131277,0.749260,0.429304,0.085426,32.000000,2.742523,1.303171,-0.063000,3.749732,3.691439,0.075252,0.563380,0.383570,7.10000,0.112140,1.450000,0.875000,9.993400,0.254194,6.125000,20.000000,30.000000,22.000000,3.007227,2.423082,3.591836,2.566016,11.809585,12.736379,12.154331,2.180266,1.420899,2.668708,1.713829,0.350011,-0.263049,0.981356,-0.069792,3.700010,3.405070,NaN,3.500016,3.700010,3.405070,NaN,3.500016,3.450010,3.230070,NaN,3.250016,0.658914,0.063256,1.351414,0.220293,1.613752,0.799833,2.043961,1.251222,2.190000,0.94000,2.65280,1.750000,8.950000,8.350000,9.550000,8.700000,0.436972,0.112315,1.225011,0.178599,1.747772,1.063624,2

Macroeconomic data: (877, 112)
------------------------------


,TradeDateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
0,20161207,2789,8478,Sell,653168.0,0.0,Unknown,1.0
1,20170329,2574,14562,Buy,1656487.0,0.0,Unknown,1.0
2,20170418,2574,4747,Buy,939673.0,0.0,Unknown,1.0


Trade data: (6762021, 8)


# 4. PREPROCESSING

## 4.1. TRADE DATA

In [44]:
# check missings
count_missings(trade)

,Total,Percent
Price,4617933,68.292201


In [45]:
trade.describe()

,TradeDateKey,CustomerIdx,IsinIdx,NotionalEUR,Price,CustomerInterest
count,6.762021e+06,6.762021e+06,6.762021e+06,6.762021e+06,2.144088e+06,6.762021e+06
mean,2.016750e+07,1.922354e+03,1.460369e+04,6.313228e+06,1.779008e+05,3.271106e-01
std,6.768082e+03,8.579293e+02,7.963809e+03,2.712408e+08,1.318503e+06,4.691581e-01
min,2.016010e+07,0.000000e+00,0.000000e+00,-1.485547e+08,-9.999990e+05,0.000000e+00
25%,2.016063e+07,1.288000e+03,7.392000e+03,3.410980e+05,9.313780e+01,0.000000e+00
50%,2.017023e+07,2.090000e+03,1.522900e+04,8.812120e+05,1.010000e+02,0.000000e+00
75%,2.017093e+07,2.574000e+03,2.211900e+04,2.136842e+06,1.075000e+02,1.000000e+00
max,2.018042e+07,3.470000e+03,2.739400e+04,2.000000e+11,9.999999e+06,1.000000e+00


In [10]:
trade["CustomerInterest"] = 1
trade["CustomerInterest"][trade["TradeStatus"] == "Holding"] = 0

In [11]:
# convert dates
trade["TradeDateKey"] = pd.to_datetime(trade["TradeDateKey"], format = '%Y%m%d')

In [12]:
### add week index

# function
def week_lag_idx(date, end_date):
    return round((end_date - date).dt.days / 7 + 0.4)

# week index
trade["WeekLag"] = week_lag_idx(trade["TradeDateKey"], date(2018, 4, 23))
test["WeekLag"] = 0

In [39]:
# aggregate weekly data
vars = ["WeekLag", "CustomerIdx", "IsinIdx", "BuySell", "CustomerInterest"]
train = trade[vars].groupby(["WeekLag", "CustomerIdx", "IsinIdx", "BuySell"], as_index = False).max()

The number of observations and bonds per customer is different in trade and test data. Trade data only contains bonds that a given customer has actually traded. In test data, for each customer, the set of bonds is only a subset of the bonds that he actually traded in the past (but not the whole set, which leads to a smaller number of observations per customer in the test data). Also, the test set is biased towards very active traders.

In [50]:
# check number of cases per customer
display(trade.groupby("CustomerIdx").IsinIdx.nunique().describe())
print("------------------------------")
display(test.groupby("CustomerIdx").IsinIdx.nunique().describe())

count     3439.000000
mean       405.832800
std       1145.875089
min          1.000000
25%          5.000000
50%         33.000000
75%        233.500000
max      14132.000000
Name: IsinIdx, dtype: float64

------------------------------


count    2495.000000
mean       97.145892
std       301.375388
min         1.000000
25%         3.000000
50%        12.000000
75%        58.000000
max      5044.000000
Name: IsinIdx, dtype: float64

In [ ]:
### ADDING MISSING COMBINATIONS

# extract all unique values
#l1 = list(trade.TradeDateKey.unique())
#l2 = list(trade.CustomerIdx.unique())
#l3 = list(trade.IsinIdx.unique())
#l4 = list(trade.BuySell.unique())

# create lists with combinations
#lp1, lp2, lp3, lp4 = pd.core.reshape.util.cartesian_product([l1, l2, l3, l4])

# convert to dataframe
#combs = pd.DataFrame(dict(TradeDateKey = lp1, CustomerIdx = lp2, IsinIdx = lp3, BuySell = lp4))

# merge missing combinations
#print(trade.shape)
#trade = trade.merge(combs, how = "right")
#print(trade.shape)

## 4.2. CUSTOMER DATA

TODO: Solve the issue of biased sampling due to the mandatory reporting before trusting the statistics!

There are 3471 unique customers in the training data of which 2495 show up in the test set. There are no unknown customers in the test set.

In [14]:
cust.CustomerIdx.nunique()

3471

In [15]:
# Compare number of test customers showing up in cust set to number of unique customers in test set
np.sum(np.in1d(test.CustomerIdx.unique(), cust.CustomerIdx.unique()))/test.CustomerIdx.nunique()

1.0

In [13]:
np.percentile(trade.groupby("CustomerIdx").size(), range(0,100,10))

array([1.0000e+00, 2.0000e+00, 5.0000e+00, 1.1000e+01, 2.5000e+01,
       5.5000e+01, 1.3600e+02, 3.6060e+02, 9.6940e+02, 3.9168e+03])

Customer information:
- 5 different sectors
- 41 subsectors
- 3 regions
- 99 countries

We can include sectors and regions directly, but may want to reduce the dimension of subsectors and countries

In [26]:
cust.columns

Index(['CustomerIdx', 'Sector', 'Subsector', 'Region', 'Country'], dtype='object')

In [28]:
cust_dummies = pd.concat([cust.CustomerIdx, pd.get_dummies(cust.Sector), pd.get_dummies(cust.Region)], axis=1)

In [30]:
cust_dummies.describe()

,CustomerIdx,Asset Managers & Hedge Funds,Asset Owners,Banks and Intermediaries,Corporation,Official Institution - OI,Americas,Asia Pacific,"Europe, Middle East and Africa"
count,3471.000000,3471.000000,3471.000000,3471.000000,3471.000000,3471.000000,3471.000000,3471.000000,3471.000000
mean,1735.000000,0.431288,0.096226,0.414002,0.018438,0.040046,0.252089,0.222126,0.525785
std,1002.135719,0.495327,0.294943,0.492620,0.134550,0.196096,0.434274,0.415735,0.499407
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,867.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1735.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,2602.500000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,3470.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Most customers are asset managers or banks/intermediaries (possibly high volume groups?). Distribution over Americas, Europe/Africa, Asia is 53/25/22

In [40]:
train = train.merge(cust_dummies, on = "CustomerIdx", how='left')
test = test.merge(cust_dummies, on = "CustomerIdx", how='left')

In [41]:
train.columns

Index(['WeekLag', 'CustomerIdx', 'IsinIdx', 'BuySell', 'CustomerInterest',
       'Asset Managers & Hedge Funds', 'Asset Owners',
       'Banks and Intermediaries', 'Corporation', 'Official Institution - OI',
       'Americas', 'Asia Pacific', 'Europe, Middle East and Africa'],
      dtype='object')

In [42]:
train[['Asset Managers & Hedge Funds', 'Asset Owners',
       'Banks and Intermediaries', 'Corporation', 'Official Institution - OI']].mean()

Asset Managers & Hedge Funds    0.735066
Asset Owners                    0.056058
Banks and Intermediaries        0.199650
Corporation                     0.001259
Official Institution - OI       0.007966
dtype: float64

## 4.3. MACROECONOMIC DATA

In [46]:
macro.describe()

,DateKey,SSE,DAX,EUROSTOXX,VSTOXX,FTSE100,HSI,NIKKEI,DOWJONES_INDU,SP500,VIX,FX_USD.ARS,FX_USD.AUD,FX_USD.BRL,FX_USD.CAD,FX_USD.CHF,FX_USD.CNO,FX_USD.CNY,FX_USD.EUR,FX_USD.GBP,FX_USD.HKD,FX_USD.IDR,FX_USD.JPY,FX_USD.NOK,FX_USD.SGD,FX_USD.TRY,FX_USD.ZAR,MoneyMarket_ARS3M,MoneyMarket_AUD3M,MoneyMarket_CAD3M,MoneyMarket_CHF3M,MoneyMarket_CNO3M,MoneyMarket_CNY3M,MoneyMarket_EUR3M,MoneyMarket_GBP3M,MoneyMarket_HKD3M,MoneyMarket_IDR3M,MoneyMarket_JPY3M,MoneyMarket_NOK3M,MoneyMarket_SGD3M,MoneyMarket_TRY3M,MoneyMarket_USD3M,MoneyMarket_ZAR3M,Swap_ARS10Y,Swap_ARS2Y,Swap_ARS5Y,Swap_AUD10Y,Swap_AUD2Y,Swap_AUD30Y,Swap_AUD5Y,Swap_BRL10Y,Swap_BRL2Y,Swap_BRL5Y,Swap_CAD10Y,Swap_CAD2Y,Swap_CAD30Y,Swap_CAD5Y,Swap_CHF10Y,Swap_CHF2Y,Swap_CHF30Y,Swap_CHF5Y,Swap_CNH10Y,Swap_CNH2Y,Swap_CNH30Y,Swap_CNH5Y,Swap_CNO10Y,Swap_CNO2Y,Swap_CNO30Y,Swap_CNO5Y,Swap_CNY10Y,Swap_CNY2Y,Swap_CNY30Y,Swap_CNY5Y,Swap_EUR10Y,Swap_EUR2Y,Swap_EUR30Y,Swap_EUR5Y,Swap_GBP10Y,Swap_GBP2Y,Swap_GBP30Y,Swap_GBP5Y,Swap_HKD10Y,Swap_HKD2Y,Swap_HKD30Y,Swap_HKD5Y,Swap_IDR10Y,Swap_IDR2Y,Swap_IDR30Y,Swap_IDR5Y,Swap_JPY10Y,Swap_JPY2Y,Swap_JPY30Y,Swap_JPY5Y,Swap_NOK10Y,Swap_NOK2Y,Swap_NOK30Y,Swap_NOK5Y,Swap_SGD10Y,Swap_SGD2Y,Swap_SGD30Y,Swap_SGD5Y,Swap_TRY10Y,Swap_TRY2Y,Swap_TRY5Y,Swap_USD10Y,Swap_USD2Y,Swap_USD30Y,Swap_USD5Y,Swap_ZAR10Y,Swap_ZAR2Y,Swap_ZAR30Y,Swap_ZAR5Y
count,8.770000e+02,875.000000,876.000000,876.000000,875.000000,876.000000,876.000000,875.000000,876.000000,876.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,876.000000,876.000000,876.000000,877.000000,876.000000,877.000000,877.000000,877.000000,875.000000,877.000000,877.000000,877.000000,877.000000,848.000000,877.000000,877.000000,877.000000,848.000000,877.000000,877.000000,877.000000,848.000000,877.000000,877.000000,876.000000,877.000000,877.000000,876.000000,876.000000,876.000000,876.000000,877.000000,877.000000,876.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000,875.000000,877.000000,877.000000,877.000000,876.000000,877.000000,877.000000,876.000000,876.000000,876.000000,876.000000,877.000000,877.000000,877.000000,877.000000,876.000000,877.000000,877.000000,877.000000,877.000000,877.000000,877.000000
mean,2.016278e+07,3310.487480,11355.682289,3331.244218,20.305856,6873.943179,24726.487500,19148.118977,19709.190297,2256.015862,14.840485,0.075868,0.757052,0.302329,0.771900,1.025955,0.153128,0.153606,1.127517,1.391046,0.128601,0.000075,0.008848,0.122075,0.729053,0.317650,0.075063,33.804797,1.982505,1.061660,-0.743487,2.952843,2.934020,-0.217630,0.492492,0.696107,6.820818,0.013370,1.074551,1.204376,10.906146,0.915570,6.911882,14.731049,23.861161,17.227100,2.692911,1.987294,3.110268,2.284876,11.953918,11.489892,11.785195,1.930044,1.198501,2.353916,1.467808,-0.025181,-0.758534,0.567229,-0.496103,3.549205,3.032505,3.841399,3.284564,3.557818,3.037124,3.847369,3.291735,3.431292,2.973169,3.739949,3.199941,0.667481,-0.186979,1.280212,0.093212,1.375099,0.690448,1.641403,1.015341,2.031283,1.224180,2.353585,1.688991,8.927836,8.344221,9.525786,8.683634,0.249206,0.002160,0.860108,0.060174,1.766093,1.003113,2.034099,1.311117,2.465752,1.449929,2.830213,1.984970,11.074696,11.873001,11.496660,2.137045,1.320768,2.468450,1.755107,8.180788,7.216463,8.185374,7.683478
std,9.840161e+03,413.973635,1136.826936,255.117381,5.979965,515.046544,3406.147361,2008.348280,2709.444089,245.959213,4.483849,0.022469,0.026942,0.027013,0.028120,0.028106,0.005431,0.006053,0.050020,0.109302,0.000490,0.000002,0.000494,0.004979,0.018090,0.046718,0.006563,43.144767,0.255217,0.287314,0.080378,0.520162,0.539430,0.137594,0.120332,0.301233,0.922109,0

## 4.4. BONDS DATA

Check that bonds after maturity are not tradeable. If this is correct, the easiest solution would be to manually correct these to 0 in model predictions

In [52]:
bond.describe(include="all")

,IsinIdx,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
count,27411.000000,27411.000000,2.741100e+04,2.741100e+04,27411,27411,27411,27411,27411,27411,27411,27411,27406,27406,27394,2.741100e+04,27411
unique,NaN,NaN,NaN,NaN,9,23,3,8,16,38,105,29,13,338,14,NaN,6
top,NaN,NaN,NaN,NaN,SEN,USD,FLOW G10,AMERICAS,HG CASH,HG CASH NONFIN,US NONFIN 5Y AND IN CASH,NR,Financial,Commer Banks Non-US,Euro mtn,NaN,FIXED
freq,NaN,NaN,NaN,NaN,21233,16358,16552,10732,7928,5608,1714,4874,10046,3715,7374,NaN,24105
mean,13705.000000,1748.038598,2.023912e+07,2.013163e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.044049e+09,NaN
std,7913.018451,1028.719783,8.515510e+04,4.165186e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.274043e+10,NaN
min,0.000000,0.000000,2.015043e+07,1.955010e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN
25%,6852.500000,855.000000,2.019042e+07,2.012013e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+08,NaN
50%,13705.000000,1753.000000,2.022013e+07,2.014062e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.500000e+08,NaN
75%,20557.500000,2647.000000,2.025091e+07,2.016071e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000e+09,NaN


In [54]:
bond_dummies = pd.concat([bond.IsinIdx, 
                          pd.get_dummies(bond.ActivityGroup), 
                          pd.get_dummies(bond.CompositeRating)], axis=1)

In [55]:
train = train.merge(bond_dummies, on = "IsinIdx", how='left')
test = test.merge(bond_dummies, on = "IsinIdx", how='left')

## 4.5. MARKET DATA

# 5. DATA EXPORT

In [17]:
# export CSV
train.to_csv("../data/prepared/train.csv", index = False, float_format = "%.8f")
test.to_csv("../data/prepared/test.csv",   index = False, float_format = "%.8f")